# Neuron Skeletonization Service Notebook

Copyright (c) 2025 Open Brain Institute

+ Author(s): 
    - Michael W. Reimann < michael.reimann@openbraininstitute.org >
    - Marwan Abdellah < marwan.abdellah@openbraininstitute.org >
    - James Isbister < james.isbister@openbraininstitute.org >

Last modified: 12.2025

# Imports

In [ ]:
# All essential imports needed for the notebook
from IPython.display import display, HTML
from obi_auth import get_token
from obi_notebook import get_projects, get_entities, get_environment
from entitysdk.models import EMCellMesh
from entitysdk.client import Client
import os
import obi_one as obi

## Authentication and project selection
We authenticate with the OBI platform.
### Project selection
Select from the dropdown menu the project that the output (a morphology with extracted spines) should be registered to. It will be available only in that project context. 

The widget lists all projects you have access to.

In [ ]:
# environment = get_environment.get_environment()
environment = "staging"
token = get_token(environment=environment)
os.environ["OBI_AUTHENTICATION_TOKEN"] = token
project_context = get_projects.get_projects(token, env=environment)

## Display EMCellMesh table

Input into the skeletonization process is a mesh describing the surface of a neuron. 

Below we will display a list of all surface meshes you have access to in the project context you selected above. This is a combination of publicly available entities open to all users and cell meshes you or your team have registered inside the project.
You can filter the list based on the EM dataset the meshes originate from. You can also search for specific neuron identifiers (typically called "pt_root_id").

Please select one mesh from the list below.

In [ ]:
mesh_ids = []
mesh_ids = get_entities.get_entities(
    'em-cell-mesh', token, [], env=environment, project_context=project_context, 
    multi_select=True, page_size=20, add_columns=["dense_reconstruction_cell_id"])

## Check the selection 

In [ ]:
if len(mesh_ids) == 0:
    display(HTML("<h3 style='color:#EDB95E'>⚠️ No mesh selected</h3>"))
    raise SystemExit()

Prepare

# Create and execute GridScanGenerationTask and SkeletonizationTask

In [ ]:
cell_mesh_from_ids = [obi.EMCellMeshFromID(id_str=mesh_id) for mesh_id in mesh_ids]

skeletonization_scan_conf = obi.SkeletonizationScanConfig(info=obi.Info(campaign_name="Skeletonization test campaign", campaign_description="Skeletonization test description"),
                                                     initialize=obi.SkeletonizationScanConfig.Initialize(
                                                                      cell_mesh=cell_mesh_from_ids, 
                                                                      neuron_voxel_size=0.1, 
                                                                      spines_voxel_size=0.05,
                                                                      segment_spines=True,))

grid_scan_generation_task = obi.GridScanGenerationTask(form=skeletonization_scan_conf, output_root='../../../obi-output/skeletonization/grid_scan')

db_client = Client(environment=environment, token_manager=token, project_context=project_context)
grid_scan_generation_task.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan_generation_task, db_client=db_client)